In [1]:
from pathlib import Path
import pandas as pd
import torch
import hvplot.pandas

import diffhydro as dh
import diffhydro.pipelines as dhp

from diffhydro.pipelines import CalibrationRouter, CalibrationModule
from diffhydro.utils import SimpleTimeSeriesSampler

from utils.io import load_single_vpu

### Experiment parameters

In [2]:
plength_thr=10**4
node_thr=10**4
max_delay = 32
dt = 1/24

pred_len = 2048
n_epoch = 15
device = "cuda:7"
vpu = "305"

root = Path("./data")

### Loading data

In [3]:
runoff, discharge, g = load_single_vpu(root, vpu, device=device, 
                                       plength_thr=plength_thr, node_thr=node_thr)

#### Upstream stats computations ... ####


Computing breakpoints:   0%|          | 0/15562 [00:00<?, ?it/s]

#### Segmentation into subgraphs ... ####
Removing edges...


  0%|          | 0/15562 [00:00<?, ?it/s]

Segment graph into connected components....
Build subgraphs for each cluster and node-cluster map...


...

  0%|          | 0/19 [00:00<?, ?it/s]

Computing merged graphs node idxs...


  0%|          | 0/19 [00:00<?, ?it/s]

Match breakpoint nodes across clusters...


  0%|          | 0/19 [00:00<?, ?it/s]

In [4]:
tr_data = SimpleTimeSeriesSampler(runoff.sel(time=slice(None, "1979")), 
                                  discharge.sel(time=slice(None, "1979")), 
                                  init_len=max_delay, 
                                  pred_len=pred_len)

te_data = SimpleTimeSeriesSampler(runoff.sel(time=slice("1980",None)), 
                                  discharge.sel(time=slice("1980", None)), 
                                  init_len=max_delay, 
                                  pred_len=pred_len)

### Standard calibration

In [ ]:
model = CalibrationRouter(g, max_delay, dt).to(device)
calib = CalibrationModule(model, tr_data, te_data).to(device)

te_nse, tr_nse = calib.callibrate(n_epoch=n_epoch)

(te_nse.hvplot(label="Test") *\
 tr_nse.hvplot(label="Train")).opts(ylim=(.999, 1))

### Handling very large graphs

- For very large graphs, the GPU memory constraint may not allow to callibrate the full network parameters in one go.
- In this case, the CalibrationModule provides helpers to sequentially calibrate sub-basins from upstream to downstream.

In [6]:
model = CalibrationRouter(g, max_delay, dt).to(device)
calib = CalibrationModule(model, tr_data, te_data).to(device)

te_nse, tr_nse = calib.calibrate_staged_sequentially(n_epoch=10)

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

...

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

In [11]:
(te_nse.hvplot(label="Test") *\
 tr_nse.hvplot(label="Train")).opts(ylim=(.999, 1))

AttributeError: 'tuple' object has no attribute 'hvplot'